In [1]:
!pip install unidecode twython tweepy beautifulsoup4 tika

     |████████████████████████████████| 235 kB 5.2 MB/s 
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32891 sha256=01dee4289e941aee5c240ee8d4db70f2be556ab2843b44ecdcc39a62ffc23c3c
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


In [2]:
import warnings; warnings.simplefilter('ignore')
import tweepy, json, nltk, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs

In [3]:
Ck = 'gchnKJTh8AGsFnrvJgM7k76V4' # consumer_key
Cs = '6uknHWCJKaHeboqhPtue6cSpG4n6oa0K7ysj6Myp9OYGKU809q' # consumer_secret
At = '1437586719935631363-Syv39uMo0EayIwDPHaZcnX00kaQdjJ' # access_token
As = 'X5TgXGdd48Yhm0lcoweyekqH1pITlktpCoQJ3ch5LSmHL' # access_secret
'Done'

'Done'

In [4]:
def twitter_connect(Ck, Cs, At, As, verbose = 0):
    try:
        auth = tweepy.OAuthHandler(Ck, Cs)
        auth.set_access_token(At, As)
        twitter = tweepy.API(auth, timeout=120)
        if verbose != 0:
            user = twitter.verify_credentials()
            print('Welcome "%s" you are now connected to twitter server' %user.name)
        return twitter
    except:
        print("Connection failed, please check your API keys or connection")
        return None

In [5]:
# Koneksi ke twitter
twitter = twitter_connect(Ck, Cs, At, As, verbose = 1)

Welcome "Renata Suyudi" you are now connected to twitter server


In [32]:
# Max 30000 tweet per "API call"
topic = 'covid'
N = 100 # jumlah tweet yang ingin diambil
bahasa = 'id'
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
tweets = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(tweets)))

 Berhasil mendapatkan 100 tweets


In [33]:
# Next data
last_id = T[-1]._json['id'] - 1
last_id
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended', max_id=last_id)
tweets.extend([t._json for t in T])

print('Jumlah data sekarang = ', len(tweets))

Jumlah data sekarang =  200


In [34]:
# Datanya berbentuk JSON
tweets[0]['created_at'], tweets[0]['full_text'], 

('Mon Nov 01 15:20:55 +0000 2021',
 'RT @AtiqZaim: Banyak iktibar yang kita boleh ambil dari COVID, antaranya:\n\n1. Hidup ini terlalu singkat\n2. Pekerjaan kita hanya sementara\n3…')

In [35]:
def saveTweets(tweets, file='covid.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [36]:
# Menyimpan hasil crawling twitter
fileName = 'datacovid.json'
saveTweets(tweets,file=fileName)
print('Saved to '+fileName)

Saved to datacovid.json


In [37]:
import pandas as pd

In [40]:
data = pd.read_json("datacovid.json", lines=True)
print(data)

                   created_at  ...  quoted_status
0   2021-11-01 15:20:55+00:00  ...            NaN
1   2021-11-01 15:20:53+00:00  ...            NaN
2   2021-11-01 15:20:51+00:00  ...            NaN
3   2021-11-01 15:20:51+00:00  ...            NaN
4   2021-11-01 15:20:50+00:00  ...            NaN
..                        ...  ...            ...
195 2021-11-01 15:10:55+00:00  ...            NaN
196 2021-11-01 15:10:54+00:00  ...            NaN
197 2021-11-01 15:10:47+00:00  ...            NaN
198 2021-11-01 15:10:45+00:00  ...            NaN
199 2021-11-01 15:10:44+00:00  ...            NaN

[200 rows x 31 columns]


In [46]:
# Max 30000 tweet per "API call"
topic = 'covid'
N = 100 # jumlah tweet yang ingin diambil
bahasa = 'id'
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
tweets = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(tweets)))

 Berhasil mendapatkan 100 tweets


In [47]:
# Next data
last_id = T[-1]._json['id'] - 1
last_id
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended', max_id=last_id)
tweets.extend([t._json for t in T])

print('Jumlah data sekarang = ', len(tweets))

Jumlah data sekarang =  200


In [48]:
# Datanya berbentuk JSON
tweets[0]['created_at'], tweets[0]['full_text'], 

('Mon Nov 01 15:21:38 +0000 2021',
 'RT @redzuanNewsMPB: Kit ujian kendiri COVID-19 kini boleh didapati pada harga serendah RM6.90 di pasaraya, kedai serbaneka, stesen minyak t…')

In [49]:
def saveTweets(tweets, file='datapandemi.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [50]:
# Menyimpan hasil crawling twitter
fileName = 'datapandemi.json'
saveTweets(tweets,file=fileName)
print('Saved to '+fileName)

Saved to datapandemi.json


In [51]:
def loadTweets(file='datapandemi.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [52]:
# Me-load kembali jika (misal) analisa ingin dilakukan di lain waktu
# Sengaja nama variabelnya saya bedakan (T2)
T2 = loadTweets(file='datapandemi.json')
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "balbalpingu" : "RT @redzuanNewsMPB: Kit ujian kendiri COVID-19 kini boleh didapati pada harga serendah RM6.90 di pasaraya, kedai serbaneka, stesen minyak t…"


In [53]:
import pandas as pd

In [54]:
datapandemi = pd.read_json("datapandemi.json", lines=True)
print(datapandemi)

                   created_at  ...  quoted_status
0   2021-11-01 15:21:38+00:00  ...            NaN
1   2021-11-01 15:21:36+00:00  ...            NaN
2   2021-11-01 15:21:36+00:00  ...            NaN
3   2021-11-01 15:21:32+00:00  ...            NaN
4   2021-11-01 15:21:21+00:00  ...            NaN
..                        ...  ...            ...
195 2021-11-01 15:11:26+00:00  ...            NaN
196 2021-11-01 15:11:23+00:00  ...            NaN
197 2021-11-01 15:11:23+00:00  ...            NaN
198 2021-11-01 15:11:21+00:00  ...            NaN
199 2021-11-01 15:11:20+00:00  ...            NaN

[200 rows x 31 columns]
